<a href="https://colab.research.google.com/github/MatchLab-Imperial/deep-learning-course/blob/master/10_Data_Classification_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏠 Data collection and classification Challenge — YOLOv8

**Objective:** You have been hired as an AI expert and your new boss, who knows nothing about AI, asked you to quickly develop a DL model that can classify roofs in the UK from aerial views. The input is in the form of image crops from Google satellite views, each containing one building. Some crops may contain a few buildings but it can be considered as noise. You should collect data and finetune `yolov8n-cls.pt`  model.   Classify aerial roof images into **flat** or **pitched**. These are the constraints, otherwise you are free to make your own design choices.

- Tune training hyperparameters  
- Add more images to improve your model  
- Experiment with augmentation or optimization settings  

When you are satisfied, submit the annotated data `*CID*_data.zip` with the same file structure as the provided sample as well as your best `*CID*_best.pt` checkpoint, which will be evaluated on a hidden test set. Keep the filenames of the existing samples as they are.

---

### 📁 Expected dataset structure
<pre>
.../aerial
├─ train/
│ ├─ flat/
│ └─ pitched/
├─ val/
│ ├─ flat/
│ └─ pitched/
└─ test/
  ├─ flat/
  └─ pitched/
</pre>

---

### 📌 Notes

- You will train only the classification model `yolov8n-cls.pt`.

- Image size (`imgsz`):  
  - YOLOv8 automatically resizes any input to the `imgsz` you set in `model.train(...)`.  
  - Tip: using sizes that are multiples of 32 (224, 256, 320, 384, …) aligns well with the backbone. Larger sizes train slower, but can perform better.
  - When collecting images, there’s no strict size rule; just aim for **clear roofs**.

- Collecting more images:
  - Sources: **Google Earth**, **OpenAerialMap**, **Mapillary**, **Kaggle**, public GIS/satellite portals, or your **own drone/photos**.  
  - Prefer images that aren’t tiny/blurry/over-compressed. RGB `.jpg/.jpeg/.png/.bmp/.webp` are all fine.

- Adding new data to `/aerial`:
  1. Label by roof type and drop files into the correct folders locally:
     ```
     aerial/
     ├─ train/
     │  ├─ flat/
     │  └─ pitched/
     ├─ val/
     │  ├─ flat/
     │  └─ pitched/
     └─ test/
        ├─ flat/
        └─ pitched/
     ```
  2. Do **not** duplicate the same image across `train/`, `val/`, and `test/`.
  3. Compress local folder to [aerial.zip](https://github.com/MatchLab-Imperial/deep-learning-course/blob/master/asset/10_Classification_YOLO/aerial.zip) to upload to colab in the code block below.



In [ ]:
# !pip install -q ultralytics  # uncomment if not installed

from ultralytics import YOLO
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import random, torch

from google.colab import files
uploaded = files.upload()  # choose aerial.zip

In [ ]:
!unzip -q -o aerial.zip -d /content/
!find /content/aerial -maxdepth 2 -type d -print  # quick check

In [ ]:
DATA_ROOT = Path("/content/aerial")
assert (DATA_ROOT/"train").exists() and (DATA_ROOT/"val").exists(), "train/val folders missing"

# List classes from train/
classes = sorted([p.name for p in (DATA_ROOT/"train").iterdir() if p.is_dir()])
print("Classes:", classes)

# Collect sample images
exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
samples = []
for cls in classes:
    samples += [p for p in (DATA_ROOT/"train"/cls).iterdir() if p.suffix.lower() in exts]

random.shuffle(samples)
show = samples[:6]

# Plot
cols = 3
rows = 2
plt.figure(figsize=(12, 6))
for i, p in enumerate(show, 1):
    img = Image.open(p)
    plt.subplot(rows, cols, i)
    plt.imshow(img)
    plt.title(p.parent.name)
    plt.axis("off")
plt.tight_layout()
plt.show()

## 🛠 Training Tips — Things You Can Try

You may experiment with these arguments in `model.train()`:

| Parameter | Meaning | Suggested values |
|-----------|---------|------------------|
| `imgsz` | input image size | `224`, `320`, `384` |
| `epochs` | training duration | `10–60` |
| `batch` | batch size | `16–128` |
| `auto_augment` | built-in augmentation | `"randaugment"`, `"ta_wide"` |
| `mixup` / `cutmix` | label mixing | `0.0–0.3` |
| `erasing` | random erasing | `0.0–0.7` |
| `optimizer` | optimizer choice | `"SGD"` or `"AdamW"` |
| `cos_lr` | cosine LR schedule | `True` or `False` |

Tip: only change one or two things at a time, and always monitor `top1_acc`. (`top5_acc` is printed automatically in YOLO classification tasks, but is useless in our two-class task.)


In [ ]:
DATA_ROOT = Path("/content/aerial")
assert (DATA_ROOT/"train").exists(), "train folder missing"
assert (DATA_ROOT/"val").exists(), "val folder missing"
assert (DATA_ROOT/"test").exists(), "test folder missing"

device = 0 if torch.cuda.is_available() else "cpu"

# Loading base model
model = YOLO("yolov8n-cls.pt")

# ======== TRAIN (you may edit arguments below) ========
results = model.train(
    data=str(DATA_ROOT),     # uses train/ and val/
    imgsz=320,               # try 224/320/384
    epochs=30,               # increase if val improves
    batch=64,
    device=device,
    workers=2,
    patience=10,
    auto_augment="randaugment",
    erasing=0.5,
    mixup=0.1,
    cutmix=0.1,
    # optimizer="AdamW",
    # cos_lr=True,
    verbose=False,           # <-- minimising console output
    plots=False,             # <-- skipping plots
)

best_ckpt = model.trainer.best
print("Best checkpoint to submit:", best_ckpt)

# ======== TEST (held-out split) ========
metrics = model.val(
    data=str(DATA_ROOT),
    split="test",
    imgsz=320,
    device=device,
    verbose=False,           # <-- minimising console output
)
print(f"Test: {100*metrics.top1:.4f}%")


### 💾 Saving and Submitting Your Model

**Report**:

*   Report the classification performance you achieved.
*   Present details of your training data and training parameters such that someone can reproduce your experiment.
*   Include a figure with the learning curves. 

Once training is complete, you’ll download your best-performing model checkpoint for submission.  

Follow the short prompt below to enter your **CID** — this will automatically rename your `best.pt` file (to `*CID*_best.pt`) and trigger the download. 



In [ ]:
import re, shutil

# Ensuring best_ckpt exists
assert 'best_ckpt' in globals(), "Run the training cell first to define `best_ckpt`."
src = Path(best_ckpt)
assert src.exists(), f"Checkpoint not found at: {src}"


cid = input("Enter your College ID: ").strip()
cid = re.sub(r'[^0-9]', '', cid)
if not cid:
    raise ValueError("Invalid College ID. Please use digits only (0–9).")


dst = Path(f"/content/{cid}_best.pt")
shutil.copy(src, dst)
print(f"Prepared submission file: {dst}")

# Downloading
files.download(str(dst))